# **Modelagem - Exemplo prático - Turma 2**

## **Quais pacotes vamos usar?**

In [ ]:
!pip install Jinja2

In [ ]:
import pandas as pd
import sklearn
import jinja2

## **Quais os dados?**

**Características de 3796 clientes de loja de varejo online do Reino Unido no período de  01/12/2010 a 09/12/2011**



**RFM**
*   **Recency (recência):** maior data de compra do cliente - maior data observada no conjunto de dados
*   **Frequency (frequência):** quantidade de cupons de compra únicos
* **Monetary value (valor monetário):** soma do valor gasto pelo cliente por compra



In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/rebecadieb/modelagem_restart_data_science/main/customer_data.csv").drop_duplicates()

In [ ]:
df.head(3)

In [ ]:
df.info()

In [ ]:
df.CustomerID.nunique()==df.shape[0]

In [ ]:
df.columns = [c.strip() for c in df.columns.str.lower().str.replace(' ', '_')]
df.columns

In [ ]:
df.describe()

**Selecionando as variáveis**

In [ ]:
df_var = df[['monetary_value',	'frequency', 'recency']]

## **Qual o problema?**

**Identificação de padrões em um conjunto de dados de clientes - segmentação**

## **Qual a metodologia?**

**RFM - Análise de agrupamento (K-means)**

**Pré-processamento**

In [ ]:
df_scaler = df_var.apply(lambda x: (x-x.mean())/ x.std(), axis=0)

In [ ]:
df_scaler.head(2)

In [ ]:
df_scaler.mean().round()

In [ ]:
df_scaler.std()

**Identificação da quantidade de clusters ideal**

In [ ]:
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

In [ ]:
model = KMeans()
visualizer = KElbowVisualizer(model, k=(2,15), timings= False)
visualizer.fit(df_scaler)    
visualizer.show()   

**5 clusters escolhidos**

In [ ]:
kmeans = KMeans(n_clusters=5, random_state=12012021).fit(df_scaler)

**Associando o cluster ao dataframe**

In [ ]:
df['cluster'] = kmeans.labels_

In [ ]:
df.head(2)

In [ ]:
import plotly.express as px

fig = px.scatter_3d(
    df, x='monetary_value', y='frequency', z='recency', color = 'cluster',
    title=f'Separação dos clusters'
)

fig.show()

## **O que fazemos com isso?**

**Média dos grupos (centroides)**

In [ ]:
df['mean_unit_price'] = df['monetary_value']/df['itens_total']
df['ticket'] = df['monetary_value']/df['frequency']

In [ ]:
mean_cluster = df[['monetary_value', 'frequency', 'itens_total', 
                   'recency', 'mean_unit_price', 'cluster',
                   'ticket']].groupby(['cluster']).mean()
mean_cluster

In [ ]:
df["max_date"] = pd.to_datetime(df["max_date"])

In [ ]:
df.max_date.max()-df.max_date.min()

**Quantidade de clientes**

In [ ]:
df.groupby(['cluster']).size().to_frame('count').sort_values('count', ascending=False)

**Descrição**

In [ ]:
dict_cluster = {0:'Potenciais',
1:'Afastados',
2:'Fora da curva',
3:'Fiéis',
4:'Comuns'}

df['cluster_desc'] = df['cluster'].map(dict_cluster)

In [ ]:
mean_cluster = df[['monetary_value', 'frequency', 'itens_total', 
                   'recency', 'mean_unit_price', 'cluster_desc',
                   'ticket']].groupby(['cluster_desc']).mean()
mean_cluster

**Regra 80/20**

80% dos efeitos surgem a partir de apenas 20% das causas

In [ ]:
order_df = df.sort_values('monetary_value', ascending=False)
order_df['monetary_percent'] = order_df.monetary_value/order_df.monetary_value.sum()
order_df['monetary_percent_cumsum'] = order_df.monetary_percent.cumsum()

In [ ]:
order_df.head(3)

In [ ]:
pareto = order_df[order_df.monetary_percent_cumsum<=0.8].groupby(['cluster_desc']).size().to_frame('count_id')

In [ ]:
order_df[order_df.monetary_percent_cumsum<=0.8].shape[0]/order_df.shape[0]

In [ ]:
pareto['% total'] = pareto.count_id/pareto.count_id.sum()*100
pareto['% cluster'] = pareto.count_id.div(df.groupby(['cluster_desc']).size())
pareto.sort_values('count_id', ascending=False)

**Comparativo da média do cluster com a média geral**

In [ ]:
mean_var = df[['monetary_value', 'frequency', 'itens_total', 
                   'recency', 'mean_unit_price', 'ticket']].mean().to_frame('mean_var')
mean_var

In [ ]:
mean_cluster.div(mean_var.mean_var.values).transpose().style.applymap(lambda x: 'background-color : red' if x>1 else '')

**Encaixando novos clientes nos grupos formados**

0:'Potenciais'

1:'Afastados'

2:'Fora da curva'

3:'Fiéis'

4:'Comuns

In [ ]:
new_client = pd.DataFrame({'monetary_value': [50, 10], 'frequency': [10, 2], 'recency':[2, 10]})
kmeans.predict(new_client)